# Surrounding Landscape

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:




In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()

config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year

method = 'mapbiomas'
biomass_raw = (ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51")
           .select("AGB").rename(f"ESA_CCI_{last_year}"))

# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).Or(biomes.eq(4)).rename("biome_mask")


## Surrounding mature forest biomass

- Mean biomass of surrounding mature forests (interpolated with Gaussian kernel)
- Total surrounding mature forest cover (normalized from 0-1) at a 500m distance

In [2]:
if method == 'tmf':
    transition = ee.ImageCollection('projects/JRC/TMF/v1_2023/TransitionMap_Subtypes').mosaic().clip(roi)
    mature_mask = transition.eq(10).selfMask()
    age = ee.Image(f"{data_folder}/tmf_{last_year}")

elif method == 'mapbiomas':
    lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
                .select([f"classification_{year}" for year in config.range_1985_2020])
                .byte()
                .rename([str(year) for year in config.range_1985_2020]))
    mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).selfMask().updateMask(biomes_mask)
    age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

mature_biomass = biomass_raw.updateMask(mature_mask).rename("mature_biomass")

# keep only forests that are 1500m away from any non-forest area
edge_detec = mature_mask.unmask(-1).zeroCrossing()
distance_to_forest_edge = (edge_detec.fastDistanceTransform(1000, 'pixels').sqrt() # 30m per pixel, 30 * 1000 = 30000m radius of search
    .multiply(ee.Image.pixelArea().sqrt()).toInt16().rename("dist"))

# sur_cover = mature_mask.unmask(0).focalMean(radius = 500, units = "meters").float().updateMask(age).rename("sur_cover")
sur_cover = mature_mask.unmask(0).reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).float().updateMask(age).rename("sur_cover")

## Biomass of nearest mature forest

In [ ]:
distance_gt_1000 = distance_to_forest_edge.gt(1000).selfMask()
mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000).rename("mature_biomass")

# Aggregate the high-resolution pixels into the 10 km grid
mature_biomass_10k = mature_biomass_exclude_edge.updateMask(biomes_mask).reduceResolution(
    reducer=ee.Reducer.mean(),  # Use mean to calculate the average
    maxPixels=1024,
    bestEffort=True # Use all pixels that can fit in the larger pixel
).reproject(
    crs='EPSG:4326',
    scale=10000
).toInt16().rename("mature_biomass")

Export at a 100m resolution (same as ESA CCI Biomass) to argue for the distance-to-edge effect.

In [3]:
# export_image(distance_to_forest_edge, "distance_to_forest_edge", region = roi, scale = 30)
export_image(sur_cover, "sur_cover_reduce_resol", region = roi, scale = 30)
# export_image(mature_biomass, "mature_biomass", region = roi, scale = 100)
# export_image(mature_biomass_10k, "mature_biomass_10k", scale = 10000, region = roi)

## Amazon Quarters
Mean Biomass Value Per Climatic Region

In [12]:
# Load the mature biomass image
distance_gt_1000 = distance_to_forest_edge.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_exclude_edge = mature_biomass.updateMask(distance_gt_1000).rename("mature_biomass")

# Load the individual feature collections and combine them into a single FeatureCollection
quarters = ee.FeatureCollection([
    ee.FeatureCollection(f"{data_folder}/raw/NE_outline_no_overlap").map(lambda f: f.set('sector', 1)),
    ee.FeatureCollection(f"{data_folder}/raw/NW_outline_no_overlap").map(lambda f: f.set('sector', 2)),
    ee.FeatureCollection(f"{data_folder}/raw/SW_outline_no_overlap").map(lambda f: f.set('sector', 3)),
    ee.FeatureCollection(f"{data_folder}/raw/SE_outline_no_overlap").map(lambda f: f.set('sector', 4)),
]).flatten()

ecoregions = (
    ee.FeatureCollection("RESOLVE/ECOREGIONS/2017")
    .filterBounds(roi)
    .map(lambda feature: feature.intersection(roi))
)

# Function to get mean biomass per Feature
def create_biomass_image(feature):
    mean_biomass = mature_biomass_exclude_edge.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = feature.geometry(),
        scale = 500,
        maxPixels = 1e12
    )
    return feature.set(mean_biomass)

# Apply the function to each feature in the FeatureCollection
mean_biomass_quarters = quarters.map(create_biomass_image)
mean_biomass_ecoregions = ecoregions.map(create_biomass_image)

# Helper function to create an image from a property in the feature collection
def create_property_image(feature_collection, property_name):
    return feature_collection.reduceToImage(
        properties = [property_name],
        reducer = ee.Reducer.first()
    ).rename(property_name)

# Create images for the 'mature_biomass' and 'sector' properties
quarter_biomass = create_property_image(mean_biomass_quarters, 'mature_biomass').toInt16()
quarter = create_property_image(mean_biomass_quarters, 'sector').byte()
ecoreg_biomass = create_property_image(mean_biomass_ecoregions, 'mature_biomass').toInt16()
ecoreg = create_property_image(ecoregions, 'ECO_ID').toInt16()

quarters_ecoreg_biomass = ee.Image.cat([quarter_biomass, quarter, ecoreg_biomass, ecoreg]).rename(["quarter_biomass", "quarter", "ecoreg_biomass", "ecoreg"])

In [ ]:
atla_mask = biomes.eq(4)
mature_biomass_exclude_edge = mature_biomass_exclude_edge.updateMask(atla_mask).rename("mature_biomass")
atla_ecoreg_biomass = 

In [13]:
export_image(quarters_ecoreg_biomass, "quarters_ecoreg_biomass", region = roi, scale = 5000)